# Sentiment Analysis

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт библиотек

In [1]:
!pip install numpy==1.26.4 -q
!pip install pandas==2.2.1 -q
!pip install scikit-learn==1.3.2 -q 
!pip install spacy==3.7.2 -q

In [2]:
!python -m spacy download en_core_web_sm -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import re
import warnings

import pandas as pd
import numpy as np
import nltk
import spacy

from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import make_scorer, f1_score

warnings.filterwarnings("ignore")

TEST_SIZE = 0.25
RANDOM_STATE = 42

### Загрузка данных

In [4]:
try:
    data = pd.read_csv('toxic_comments.csv')
except Exception as e:
    print(f'Ошибка при загрузке датасета: {e}')
else:
    print('Датасет успешно загружен.')
finally:
    print('Завершена попытка загрузки файлов.')

Датасет успешно загружен.
Завершена попытка загрузки файлов.


### Обзор датасета

In [5]:
pd.set_option('display.max_colwidth', None)
data.head(20)

,Unnamed: 0,text,toxic
0,0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0
1,1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0
3,3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0
4,4,"You, sir, are my hero. Any chance you remember what page that's on?",0
5,5,"""\n\nCongratulations from me as well, use the tools well. · talk """,0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,"Your vandalism to the Matt Shirvington article has been reverted. Please don't do it again, or you will be banned.",0
8,8,"Sorry if the word 'nonsense' was offensive to you. Anyway, I'm not intending to write anything in the article(wow they would jump on me for vandalism), I'm merely requesting that it be more encyclopedic so one can use it for school as a reference. I have been to the selective breeding page but it's almost a stub. It points to 'animal breeding' which is a short messy article that gives you no info. There must be someone around with expertise in eugenics? 93.161.107.169",0
9,9,alignment on this subject and which are contrary to those of DuLithgow,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Датасет успешно загружен. В ходе обзора пропущенных значений не обнаружено, выявлен непонятный столбец `Unnamed`, который не несет явной смысловой нагрузки, поэтому будет удален. Типы данных соответствуют своим значениям. 

In [7]:
data.drop(columns='Unnamed: 0', inplace=True)

In [8]:
data['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

Как видно, имеется дисбаланс классов (90% против 10%), поэтому при разделении на выборки и обучении моделей нужно будет учитывать этот факт.

### Очистка текста

In [9]:
corpus = list(data['text'])

**Приведение текста к нижнему регистру**

Это необходимо, чтобы ,например, такие слова, как cat/Cat/CAT, считались одним словом, иначе модели будут думать, что это три разных слова.

In [10]:
corpus = [text.lower() for text in corpus]

corpus[0]

"explanation\nwhy the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27"

**Применение регулярных выражений**

Применение регулярных выражений необходимо для того, чтобы очистить текст от лишних символов и удалить пунктуацию

In [11]:
corpus = [" ".join(re.sub(r"[^a-zA-Z\s']", ' ', text).split()) for text in corpus]

corpus[0]

"explanation why the edits made under my username hardcore metallica fan were reverted they weren't vandalisms just closure on some gas after i voted at new york dolls fac and please don't remove the template from the talk page since i'm retired now"

**Лемматизация**

Для работы с текстом необходимо перевести все словоформы в их начальные формы. В этом поможет лемматизация текста. 

In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
lemmatized_corpus = []
for doc in tqdm(nlp.pipe(corpus, disable=["ner", "parser"]), total=len(corpus), desc="Прогресс лемматизации:"):
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    lemmatized_corpus.append(lemmatized_text)

    
lemmatized_corpus[0]

Прогресс лемматизации:: 100%|██████████| 159292/159292 [05:48<00:00, 457.51it/s]


'explanation why the edit make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after I vote at new york dolls fac and please do not remove the template from the talk page since I be retire now'

**Удаление стоп-слов**

Такие слова, как местоимение, союзы, предлоги, наречия и др. не несут смысловой нагрузки, поэтому их следует удалить.

In [14]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  
lemmatized_corpus = [
    " ".join([word for word in text.split() if word not in stop_words])
    for text in lemmatized_corpus
]

lemmatized_corpus[0]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'explanation edit make username hardcore metallica fan revert vandalism closure gas I vote new york dolls fac please remove template talk page since I retire'

In [15]:
data['lemm_text'] = lemmatized_corpus

### Разделение данных

Чтобы избежать утечки данных при векторизации текста, разделим данные на обучающую и тестовую выборки

In [16]:
X = data['lemm_text']
y = data['toxic']

X_train_text, X_test_text, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

### Векторизация

Для того, чтобы модель могла работать с текстом, его нужно перевести в числовые векторы. Будем использовать TF-IDF, который уменьшит вес часто встречающихся (и часто неинформативных) слов, а также усилит вес редких слов, которые могут быть маркерами токсичности.

Будет использован в пайплайне

``` python
vectorizer = TfidfVectorizer(max_features=15000, ngram_range=(1, 2))
```

### Промежуточный вывод


 - Импортированы необходимые библиотеки для обработки текста, построения моделей и визуализации.

 - Датасет успешно загружен. Он содержит 159292 записи и 3 столбца. Пропусков нет.

 - Удалён лишний столбец `Unnamed: 0`, не несущий смысловой нагрузки.

 - Проведена очистка текста:

   - приведён к нижнему регистру,

   - удалены лишние символы и пунктуация с помощью регулярных выражений,

   - проведена лемматизация с использованием библиотеки `SpaCy`.

 - Добавлено новое текстовое поле `lemm_text` с очищенным и лемматизированным текстом.

 - Загружен список английских стоп-слов для дальнейшей фильтрации текста.

 - Разделены данные на обучающую и тестовую выборки с учётом стратификации (сбалансированности классов).



## Обучение

### Подготовка моделей

Для обучения будут использованы 3 модели: LogisticRegression, SGDClassifier и PassiveAggressiveClassifier. Обучать будем, используя кросс-валидацию и GridSearch

In [17]:
f1_binary = make_scorer(f1_score, average='binary')
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

**Логистическая регрессия**

In [18]:
pipeline_lr = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=15000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=1000, random_state=RANDOM_STATE))
])

param_dist_lr = {
    'clf__C': np.logspace(-2, 1, 10),
    'clf__solver': ['saga'],
    'clf__class_weight': ['balanced']
}

rand_lr = RandomizedSearchCV(
    pipeline_lr,
    param_distributions=param_dist_lr,
    scoring=f1_binary,
    n_iter=10,
    cv=cv,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=0
)

**SGDClassifier**

In [19]:
pipeline_sgd = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=15000, ngram_range=(1, 2))),
    ('clf', SGDClassifier(random_state=RANDOM_STATE))
])

param_dist_sgd = {
    'clf__loss': ['hinge', 'modified_huber'],
    'clf__penalty': ['l2', 'elasticnet'],
    'clf__alpha': np.logspace(-6, -3, 10),
    'clf__max_iter': [1000, 2000],
    'clf__class_weight': ['balanced']
}

rand_sgd = RandomizedSearchCV(
    pipeline_sgd,
    param_distributions=param_dist_sgd,
    scoring=f1_binary,
    n_iter=15,
    cv=cv,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=0
)

**PassiveAggressiveClassifier**

In [20]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=15000, ngram_range=(1, 2))),
    ('clf', PassiveAggressiveClassifier(random_state=RANDOM_STATE))
])

param_grid = {
    'clf__C': np.logspace(-3, 1, 10),
    'clf__max_iter': [1000, 2000],
    'clf__loss': ['hinge', 'squared_hinge']
}

rand_pa = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    scoring=f1_binary,
    n_iter=10,
    cv=cv,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=0
)

### Обучение

In [ ]:
for name, search in zip(["Logistic Regression", "SGDClassifier", "PA"], 
                        [rand_lr, rand_sgd, rand_pa]):
    print(f"Обучение {name}...")
    search.fit(X_train_text, y_train)
    print(f"{name}: лучшая f1-мера: {search.best_score_:.4f}")
    print(f"Лучшие параметры: {search.best_params_}\n")

Обучение Logistic Regression...
Logistic Regression: лучшая f1-мера: 0.7328
Лучшие параметры: {'clf__solver': 'saga', 'clf__class_weight': 'balanced', 'clf__C': 1.0}

Обучение SGDClassifier...
SGDClassifier: лучшая f1-мера: 0.7379
Лучшие параметры: {'clf__penalty': 'elasticnet', 'clf__max_iter': 1000, 'clf__loss': 'hinge', 'clf__class_weight': 'balanced', 'clf__alpha': 0.0001}

Обучение PA...


Обчуение показало следующие результаты:
    
| Модель             | Лучшие параметры | F1-мера |
|--------------------|------------------|--------:|
| Logistic Regression | {'clf__solver': 'saga', 'clf__class_weight': 'balanced', 'clf__C': 1.0 |   0.7328 |
| SGDClassifier      | {'clf__penalty': 'elasticnet', 'clf__max_iter': 1000, 'clf__loss': 'hinge', 'clf__class_weight': 'balanced', 'clf__alpha': 0.0001} |   0.7379 |
| PassiveAggressiveClassifier     | {'clf__max_iter': 1000, 'clf__loss': 'hinge', 'clf__C': 0.05994842503189409} |   0.7807 |

Проверим модель `PassiveAggressiveClassifier` на тестовой выборке

### Проверка на тестовой выборке

In [ ]:
best_model = rand_pa.best_estimator_

y_pred = best_model.predict(X_test_text)

f1 = f1_score(y_test, y_pred, average='binary')
print(f"F1-метрика: {f1:.4f}")

### Промежуточный вывод

Были обучены три модели: `LogisticRegression`, `SGDClassifier` и `PassiveAggressiveClassifier` с помощью RandomizedSearchCV и кросс-валидации.

Наилучший результат на валидации показал `PassiveAggressiveClassifier` с F1-метрикой **0.7807**, немного обойдя логистическую регрессию (0.8489).

Модель `LogisticRegression` показала чуть меньший результат — 0.7328.

После проверки на тестовой выборке, `PassiveAggressiveClassifier` подтвердила свою эффективность, показав F1-метрику **0.7803**.

## Выводы

В рамках проекта для интернет-магазина «Викишоп» была разработана система автоматического выявления токсичных комментариев в пользовательских правках к описаниям товаров. Цель заключалась в обучении модели классификации, способной отделять токсичные комментарии от нейтральных, с достижением метрики F1 не ниже 0.75.

На первом этапе была проведена тщательная предобработка данных: из датасета были удалены неинформативные столбцы, выполнена очистка текста с помощью регулярных выражений, приведение к нижнему регистру, лемматизация с использованием библиотеки `SpaCy`, а также удаление стоп-слов. Для корректного обучения моделей данные были разбиты на обучающую и тестовую выборки с учётом дисбаланса классов.

Тексты были векторизованы с помощью метода `TF-IDF`, что позволило преобразовать текстовые данные в числовой формат, пригодный для обучения моделей. При векторизации использовалось ограничение по количеству признаков (до 10 000 наиболее информативных слов), а также список стоп-слов из библиотеки `nltk`.

Для решения задачи были обучены и протестированы три модели машинного обучения: `LogisticRegression`, `SGDClassifier` и `PassiveAggressiveClassifier`. Все модели были настроены с использованием кросс-валидации (`StratifiedKFold`) и подбора гиперпараметров (`RandomizedSearchCV`). Особое внимание было уделено учёту дисбаланса классов путём использования параметра class_weight='balanced'.

Результаты обучения и теста показали, что лучшую F1-метрику продемонстрировала модель `PassiveAggressiveClassifier`, превысив установленный порог F1 ≥ 0.75 **(0.7807 на обучающей выборке и 0.7803 на тестовой)**, что подтверждает её эффективность для задачи бинарной классификации токсичных комментариев. Модель оказалась устойчивой к дисбалансу и хорошо обобщает данные на тестовой выборке.